### 01. 라이브러리 임포트

In [2]:
from sklearn.naive_bayes import GaussianNB

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

### 02. 데이터 불러오기 (xlsx)

In [3]:
df = pd.read_excel('./data/wpdb.xlsx')
df.head()

,UID,CenterId,StudentName,SchoolName,SchoolGrade,SchoolLevel,StudentGrade,Address,Address2,StudentStatus,cs_count,max_sche_date,withdraw_csl_count,remain_csl_count,supply_count,clinic_count,abs_count,pre_count,target
0,00249736-53A0-4D68-8CB9-DFA306A33435,15,임현승,성동중학교,1학년,high,17,NaN,NaN,drop,5,2023-11-17,0,3,0,0,1,41,0
1,00515C99-00A6-444B-93A3-73D7BB0852E5,15,노해율,내성중학교,3학년,middle,16,부산 동래구 금정마을로 135-9(동래 래미안 아이파크),2단지-5호차,present,5,2023-07-20,0,19,0,0,0,52,0
2,00600FC7-C7DE-40DE-AD7D-A1D94DD5D212,16,신아인,경일중학교,3학년,middle,16,부산 강서구 명지국제7로 110(더 힐 시그니처),NaN,present,8,2024-02-15,0,0,0,0,0,0,1
3,0063DEBD-095C-4804-A246-7EDD5160419A,14,손태윤,브니엘예술중학교,3학년,middle,16,NaN,NaN,drop,2,2023-10-01,0,0,0,0,0,6,0
4,007378EC-9A44-4A75-8B1A-70D13FBE7832,16,오주안,오션중학교,3학년,middle,16,NaN,명지 오션 11로 87,present,29,2024-02-15,0,23,0,0,1,278,1


### 03. 학습에 사용될 데이터와 예측 데이터 설정
- 학습 데이터 X
- 예측 데이터 y
    - 0과 1의 값만 존재하며, 분류를 이용하여 예측 예정

In [4]:
X=df[df.columns[6:]]
X.head()

,StudentGrade,Address,Address2,StudentStatus,cs_count,max_sche_date,withdraw_csl_count,remain_csl_count,supply_count,clinic_count,abs_count,pre_count,target
0,17,NaN,NaN,drop,5,2023-11-17,0,3,0,0,1,41,0
1,16,부산 동래구 금정마을로 135-9(동래 래미안 아이파크),2단지-5호차,present,5,2023-07-20,0,19,0,0,0,52,0
2,16,부산 강서구 명지국제7로 110(더 힐 시그니처),NaN,present,8,2024-02-15,0,0,0,0,0,0,1
3,16,NaN,NaN,drop,2,2023-10-01,0,0,0,0,0,6,0
4,16,NaN,명지 오션 11로 87,present,29,2024-02-15,0,23,0,0,1,278,1


In [5]:
X = X.drop(columns = ['Address','Address2', 'max_sche_date', 'StudentStatus'])
X

,StudentGrade,cs_count,withdraw_csl_count,remain_csl_count,supply_count,clinic_count,abs_count,pre_count,target
0,17,5,0,3,0,0,1,41,0
1,16,5,0,19,0,0,0,52,0
2,16,8,0,0,0,0,0,0,1
3,16,2,0,0,0,0,0,6,0
4,16,29,0,23,0,0,1,278,1
...,...,...,...,...,...,...,...,...,...
2492,15,2,1,20,0,0,0,64,1
2493,16,9,0,0,0,0,0,0,1
2494,16,3,1,9,0,0,0,23,0
2495,17,6,0,0,0,0,0,0,1


### 04. 나이 데이터 0, 1, 2로 변환

In [6]:
X['StudentGrade'].replace({15: 0, 16: 1, 17:2}, inplace=True)

In [7]:
X

,StudentGrade,cs_count,withdraw_csl_count,remain_csl_count,supply_count,clinic_count,abs_count,pre_count,target
0,2,5,0,3,0,0,1,41,0
1,1,5,0,19,0,0,0,52,0
2,1,8,0,0,0,0,0,0,1
3,1,2,0,0,0,0,0,6,0
4,1,29,0,23,0,0,1,278,1
...,...,...,...,...,...,...,...,...,...
2492,0,2,1,20,0,0,0,64,1
2493,1,9,0,0,0,0,0,0,1
2494,1,3,1,9,0,0,0,23,0
2495,2,6,0,0,0,0,0,0,1


In [8]:
pd.DataFrame(X).describe()

,StudentGrade,cs_count,withdraw_csl_count,remain_csl_count,supply_count,clinic_count,abs_count,pre_count,target
count,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000
mean,1.080497,10.095715,0.236684,8.060473,0.010813,0.028835,2.785342,64.474169,0.567080
std,0.822076,10.497140,0.648321,9.690521,0.130807,0.447538,5.310735,80.639647,0.495579
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000
50%,1.000000,6.000000,0.000000,4.000000,0.000000,0.000000,0.000000,31.000000,1.000000
75%,2.000000,14.000000,0.000000,13.000000,0.000000,0.000000,3.000000,95.000000,1.000000
max,2.000000,62.000000,19.000000,64.000000,2.000000,11.000000,54.000000,396.000000,1.000000


In [9]:
print(X.shape)
type(X)

(2497, 9)


pandas.core.frame.DataFrame

In [10]:
# 정답(레이블) 설정
y = X['target']
y # 0과 1로 구분, 1:재원 / 0:퇴원

0       0
1       0
2       1
3       0
4       1
       ..
2492    1
2493    1
2494    0
2495    1
2496    0
Name: target, Length: 2497, dtype: int64

In [11]:
print(y.shape)

(2497,)


In [12]:
y.value_counts()

target
1    1416
0    1081
Name: count, dtype: int64

In [13]:
print(y.value_counts()/len(y))

target
1    0.56708
0    0.43292
Name: count, dtype: float64


In [14]:
X = X.drop(columns = ['target'])

In [15]:
X

,StudentGrade,cs_count,withdraw_csl_count,remain_csl_count,supply_count,clinic_count,abs_count,pre_count
0,2,5,0,3,0,0,1,41
1,1,5,0,19,0,0,0,52
2,1,8,0,0,0,0,0,0
3,1,2,0,0,0,0,0,6
4,1,29,0,23,0,0,1,278
...,...,...,...,...,...,...,...,...
2492,0,2,1,20,0,0,0,64
2493,1,9,0,0,0,0,0,0
2494,1,3,1,9,0,0,0,23
2495,2,6,0,0,0,0,0,0


### 05. 결측치 확인

In [16]:
X.isnull().sum()

StudentGrade          0
cs_count              0
withdraw_csl_count    0
remain_csl_count      0
supply_count          0
clinic_count          0
abs_count             0
pre_count             0
dtype: int64

### 06. 데이터 정규화

In [17]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler_minmax=MinMaxScaler()
scaler_standard=StandardScaler()

### 1) 학습 데이터의 정규화

In [18]:
scaler_minmax.fit(X)
X_scaled_minmax_train=scaler_minmax.transform(X)
pd.DataFrame(X_scaled_minmax_train).describe()

# 최소값(min)은 0으로, 최대값(max)은 1로 나타난 걸 확인할 수 있음

,0,1,2,3,4,5,6,7
count,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000,2497.000000
mean,0.540248,0.149110,0.012457,0.125945,0.005406,0.002621,0.051580,0.162814
std,0.411038,0.172084,0.034122,0.151414,0.065404,0.040685,0.098347,0.203635
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.016393,0.000000,0.015625,0.000000,0.000000,0.000000,0.010101
50%,0.500000,0.081967,0.000000,0.062500,0.000000,0.000000,0.000000,0.078283
75%,1.000000,0.213115,0.000000,0.203125,0.000000,0.000000,0.055556,0.239899
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
'''X_scaled_standard_train'''

'X_scaled_standard_train'

### 2) 검증 데이터의 정규화

In [20]:
'''scaler_standard.fit(X_val)
X_scaled_standard_train=scaler_standard.transform(X_val)
pd.DataFrame(X_scaled_standard_train).describe()'''

'scaler_standard.fit(X_val)\nX_scaled_standard_train=scaler_standard.transform(X_val)\npd.DataFrame(X_scaled_standard_train).describe()'

### 3) 테스트 데이터의 정규화

In [21]:
'''X_scaled_minmax_test=scaler_minmax.transform(X_test)
pd.DataFrame(X_scaled_minmax_test).describe()'''

'X_scaled_minmax_test=scaler_minmax.transform(X_test)\npd.DataFrame(X_scaled_minmax_test).describe()'

### 07. 학습 데이터와 검증용 데이터 분리
- Train/Test set 분리
- Train/Validation/Test set 분리

In [22]:
# Train/Test set 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = 1) 

# stratify : 훈련 데이터와 테스트 데이터를 구분할 때, 레이블의 범주비율에 맞게 하라는 것 (회귀에는 사용 X)

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1997, 8), (500, 8), (1997,), (500,))

In [24]:
'''# Train/Validation/Test set 분리 (6:2:2)

X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.25, stratify=y_tmp, random_state=0)'''

'# Train/Validation/Test set 분리 (6:2:2)\n\nX_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.25, stratify=y_tmp, random_state=0)'

In [25]:
'''X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape'''

'X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape'

### 08. 가우시안 나이브 베이즈 모델 학습 및 예측

In [26]:
gnb = GaussianNB()  #가우시안 나이브 베이즈 모델
gnb_fit=gnb.fit(X_train,y_train) #학습 시작

y_pred=gnb_fit.predict(X_test) #학습 결과를 테스트 데이터에 적용

In [27]:
#특정 변수의 결과 뽑기(확률)
gnb_fit.predict_proba(X_test)[[1,48,51,100]] # predict_proba 함수는 각 샘플에 대해 어느 클래스에 속할 확률을 0에서 1 사이의 값으로 돌려줌

array([[0.9220089 , 0.0779911 ],
       [0.94885475, 0.05114525],
       [0.97037955, 0.02962045],
       [0.01122759, 0.98877241]])

In [28]:
#특정 변수의 결과 뽑기(범주)
gnb_fit.predict(X_test)[[1,48,51,100]]

array([0, 0, 0, 1], dtype=int64)

In [29]:
y_pred = gnb_fit.predict(X_test)

In [30]:
y_pred
#print(len(pred1))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [31]:
print('Train_Accuracy: ', gnb_fit.score(X_train, y_train),'\n')

Train_Accuracy:  0.5453179769654481 



### 09. 테스트셋에 대한 모델 정확도 확인

In [32]:
import sklearn.metrics as mt

In [33]:
accuracy = mt.accuracy_score(y_test, y_pred)
print('Test_Accuracy: ', format(accuracy,'.2f'),'\n')

Test_Accuracy:  0.52 



### 10. 모델 학습 결과 평가 및 결과지표 확인(컨퓨전 매트릭스)

In [34]:
print('Train_Accuracy: ', gnb_fit.score(X_train, y_train),'\n')

accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)
matrix = mt.confusion_matrix(y_test, y_pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)

Train_Accuracy:  0.5453179769654481 

Accuracy:  0.52 

Recall:  0.20 

Precision:  0.84 

F1_score:  0.32 

Confusion Matrix: 
 [[205  11]
 [227  57]]


In [35]:
# Accuracy = (TP + TN) / (TP + TN + FP + FN) 수동 정확도 계산
(205+57) / matrix .sum()

0.524

### 10. K-Fold 교차검증(cross-validation)

In [36]:
# 교차검증
from sklearn.model_selection import cross_val_score, cross_validate

# 각 폴드의 스코어 
scores = cross_val_score(gnb_fit, X, y, cv = 5)
scores

array([0.54      , 0.53      , 0.53306613, 0.55511022, 0.52304609])

In [37]:
pd.DataFrame(cross_validate(gnb_fit, X, y, cv =5))

,fit_time,score_time,test_score
0,0.005680,0.001166,0.540000
1,0.002452,0.002002,0.530000
2,0.002999,0.002000,0.533066
3,0.002000,0.001999,0.555110
4,0.002007,0.001000,0.523046


In [38]:
print('교차검증 평균: ', scores.mean())

교차검증 평균:  0.5362444889779558


### 11. 사전확률(prior prob) 설정 및 accuracy 확인
- priors를 통해 가우시안 나이브 베이즈의 특정 범주의 확률을 바꿀 수 있습니다. 각 범주의 사전 확률을 알고있으면 설정을 통해 더 좋은 결과를 얻을 수 있습니다. 목표변수가 0일 확률을 0.02, 1일 확률을 0.98로 설정하여 똑같이 모형을 적합하고 혼동행렬을 계산했습니다. 마지막 범주(Y=1)가 나올 확률을 높게 한 결과, 이전에서의 결과와 달리 실제 1인 경우를 잘 맞추는 대신 실제 0을 1로 잘못 분류하는 경우가 많아졌습니다.

$a^2 + a^3 = 3$

In [39]:
from sklearn.metrics import confusion_matrix #혼동행렬

gnb2 = GaussianNB(priors=[2/100, 98/100])
fitted2 = gnb2.fit(X_train,y_train)
y_pred2 = fitted2.predict(X_test)

matrix2 = mt.confusion_matrix(y_test, y_pred2)
print('Confusion Matrix:','\n', matrix2)

Confusion Matrix: 
 [[ 69 147]
 [  9 275]]


In [40]:
# Accuracy = (TP + TN) / (TP + TN + FP + FN) 수동 정확도 계산
(69+275) / matrix2.sum()

0.688

In [41]:
gnb3 = GaussianNB(priors=[98/100, 2/100])
fitted3 = gnb3.fit(X_train,y_train)
y_pred3 = fitted3.predict(X_test)

matrix3 = mt.confusion_matrix(y_test, y_pred3)
print('Confusion Matrix:','\n', matrix3)

Confusion Matrix: 
 [[212   4]
 [252  32]]


In [42]:
# Accuracy = (TP + TN) / (TP + TN + FP + FN) 수동 정확도 계산
(212+32) / matrix3.sum()

0.488

In [43]:
gnb4 = GaussianNB(priors=[57/100, 43/100])
fitted4 = gnb4.fit(X_train,y_train)
y_pred4 = fitted4.predict(X_test)

matrix4 = mt.confusion_matrix(y_test, y_pred4)
print('Confusion Matrix:','\n', matrix4)

Confusion Matrix: 
 [[207   9]
 [227  57]]


In [44]:
# Accuracy = (TP + TN) / (TP + TN + FP + FN) 수동 정확도 계산
(207+57) / matrix4.sum()

0.528

✅ 성능 지표 확인

In [45]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.88      0.32      0.47       216
           1       0.65      0.97      0.78       284

    accuracy                           0.69       500
   macro avg       0.77      0.64      0.62       500
weighted avg       0.75      0.69      0.65       500



In [55]:
%pip install visualize 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement visuaize (from versions: none)
ERROR: No matching distribution found for visuaize


In [59]:
#분류기 성능 시각화
import visualize
# visualize.visual

visualize.visualize_grid_attention(fitted4, X, y)

TypeError: can only concatenate str (not "GaussianNB") to str